# 녹음 -> blob에 저장 -> 로드 -> STT -> blob에 저장 -> 로드 -> SUM -> 결과출력/blob저장

In [2]:
import wave
import pyaudio
import azure.cognitiveservices.speech as speechsdk
from azure.storage.blob import BlobServiceClient
from datetime import datetime
import io
import keyboard
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# 🔹 .env 파일에서 환경 변수 로드
load_dotenv()

# 🔹 환경 변수에서 Azure Storage 및 OpenAI 설정 가져오기
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_API_VERSION = "2024-08-01-preview"

# 🔹 컨테이너 설정
BLOB_AUDIO_CONTAINER = "audio-files"
STT_RESULTS_CONTAINER = "stt-results"
SUMMARIZED_RESULTS_CONTAINER = "sum-results"

# 🔹 Azure Storage 클라이언트 생성
blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)

# 🔹 Azure OpenAI 클라이언트 생성
client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT,
    api_version=AZURE_OPENAI_API_VERSION
)

# 🔹 녹음 설정
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000  # 16kHz (Azure STT 최적화)
CHUNK = 1024

def create_container_if_not_exists(container_name):
    """Azure Blob Storage 컨테이너가 없으면 생성"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(container_name)
    if not container_client.exists():
        container_client.create_container()

create_container_if_not_exists(BLOB_AUDIO_CONTAINER)
create_container_if_not_exists(STT_RESULTS_CONTAINER)
create_container_if_not_exists(SUMMARIZED_RESULTS_CONTAINER)

def record_and_upload():
    """마이크로 음성을 녹음한 후, WAV 파일로 변환하여 Azure Blob Storage에 저장"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    blob_name = f"live_audio_{timestamp}.wav"
    local_audio_path = f"./{blob_name}"
    blob_client = blob_service_client.get_blob_client(container=BLOB_AUDIO_CONTAINER, blob=blob_name)

    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

    print("실시간 녹음 시작... (Enter 또는 CTRL+C를 눌러 종료)")
    frames = []

    try:
        while True:
            data = stream.read(CHUNK)
            frames.append(data)
            if keyboard.is_pressed("enter"):
                print("녹음 종료")
                break
    except KeyboardInterrupt:
        print("녹음 종료 (CTRL+C 입력 감지됨)")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    # WAV 파일로 저장
    with wave.open(local_audio_path, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"WAV 파일 저장 완료: {local_audio_path}")

    # Azure Blob Storage에 업로드
    with open(local_audio_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)

    print(f"오디오 파일 업로드 완료: {blob_name}")
    return blob_name

def transcribe_audio(blob_name):
    """Azure STT API를 사용하여 Blob Storage에서 직접 변환"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    blob_client = blob_service_client.get_blob_client(container=BLOB_AUDIO_CONTAINER, blob=blob_name)
    blob_data = blob_client.download_blob().readall()

    speech_key = os.getenv("SPEECH_API_KEY")
    speech_region = os.getenv("SPEECH_REGION")

    if not speech_key or not speech_region:
        raise ValueError("환경 변수 SPEECH_API_KEY 또는 SPEECH_REGION이 설정되지 않았습니다.")

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language = "ko-KR"

    stream_reader = io.BytesIO(blob_data)
    audio_input_stream = speechsdk.audio.AudioConfig(filename=blob_name)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input_stream)

    print("음성을 텍스트로 변환 중...")
    result = speech_recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print(f"STT 변환 결과: {result.text}")
        return result.text
    else:
        print("음성 인식 실패")
        return None

def upload_stt_result_to_blob(text_data, audio_blob_name):
    """STT 변환된 텍스트를 Blob Storage에 저장"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    text_blob_name = audio_blob_name.replace(".wav", ".txt")
    blob_client = blob_service_client.get_blob_client(container=STT_RESULTS_CONTAINER, blob=text_blob_name)
    blob_client.upload_blob(text_data, overwrite=True)
    print(f"STT 변환된 텍스트 업로드 완료: {text_blob_name}")
    return text_blob_name

def summarize_text(text):
    """Azure OpenAI를 사용하여 STT 결과를 요약"""
    prompt = f"""
    당신은 환자와 의사의 대화를 분석하는 AI입니다.  
    아래의 대화 내용을 기반으로 핵심 정보는 유지하면서도 짧고 간결한 요약을 생성하세요.  
    ❗ 줄글이 아닌, `진단:`, `수술 계획:`, `부작용:` 등의 키워드 형식을 유지하세요.  

    입력 데이터:
    \"\"\" {text} \"\"\"  
    """

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "You are an AI assistant specialized in summarizing medical conversations."},
            {"role": "user", "content": prompt}
        ]
    )

    summary = response.choices[0].message.content.strip()
    return summary

def upload_summary_to_blob(summary_text, original_filename):
    """요약된 텍스트를 Blob Storage(sum_results 컨테이너)에 저장"""
    summary_blob_name = original_filename.replace(".txt", "_summary.txt")
    blob_client = blob_service_client.get_blob_client(container=SUMMARIZED_RESULTS_CONTAINER, blob=summary_blob_name)
    blob_client.upload_blob(summary_text, overwrite=True)
    print(f"📤 요약된 텍스트 업로드 완료: {summary_blob_name}")

# 🔹 실행 흐름
audio_filename = record_and_upload()
stt_text = transcribe_audio(audio_filename)

if stt_text:
    stt_filename = upload_stt_result_to_blob(stt_text, audio_filename)
    summarized_text = summarize_text(stt_text)
    upload_summary_to_blob(summarized_text, stt_filename)

    print("\n📌 요약된 내용:")
    print(summarized_text)


실시간 녹음 시작... (Enter 또는 CTRL+C를 눌러 종료)
녹음 종료
WAV 파일 저장 완료: ./live_audio_20250203_205024.wav
오디오 파일 업로드 완료: live_audio_20250203_205024.wav
음성을 텍스트로 변환 중...
STT 변환 결과: 담론은 쓸개주머니라고도 하는데 담즙을 저장하는 역할을 하는 기관이에요. 이 담즙이라는 건 지방을 소환하는 데 도움을 주는 소화액인데 담낭을 제거한다고 해서 소화가 안 되는 건 아니에요. 담즙은 담낭에서 만드는 게 아니고 관에서 만드는 거기 때문에 담낭에 없어도 담즙은 계속 만들어 내거든요.
STT 변환된 텍스트 업로드 완료: live_audio_20250203_205024.txt
📤 요약된 텍스트 업로드 완료: live_audio_20250203_205024_summary.txt

📌 요약된 내용:
- 용어 설명: 담낭은 쓸개주머니라고도 하며, 담즙을 저장하는 기관입니다.
- 소화 기능: 담즙은 지방 소화에 도움을 주는 소화액입니다.
- 담즙 생성: 담즙은 담낭이 아닌 관에서 생성되므로, 담낭이 없어도 계속 생성됩니다.


# 로컬 저장 없이 시도 -> 오 된듯?

In [20]:
import wave
import pyaudio
import azure.cognitiveservices.speech as speechsdk
from azure.storage.blob import BlobServiceClient
from datetime import datetime
import io
import keyboard
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# 🔹 .env 파일에서 환경 변수 로드
load_dotenv()

# 🔹 환경 변수에서 Azure Storage 및 OpenAI 설정 가져오기
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_API_VERSION = "2024-08-01-preview"

# 🔹 컨테이너 설정
BLOB_AUDIO_CONTAINER = "audio-files"
STT_RESULTS_CONTAINER = "stt-results"
SUMMARIZED_RESULTS_CONTAINER = "sum-results"

# 🔹 Azure Storage 클라이언트 생성
blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)

# 🔹 Azure OpenAI 클라이언트 생성
client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_DEPLOYMENT,
    api_version=AZURE_OPENAI_API_VERSION
)

# 🔹 녹음 설정
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000  # 16kHz (Azure STT 최적화)
CHUNK = 1024

def create_container_if_not_exists(container_name):
    """Azure Blob Storage 컨테이너가 없으면 생성"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(container_name)
    if not container_client.exists():
        container_client.create_container()

create_container_if_not_exists(BLOB_AUDIO_CONTAINER)
create_container_if_not_exists(STT_RESULTS_CONTAINER)
create_container_if_not_exists(SUMMARIZED_RESULTS_CONTAINER)

def record_and_upload():
    """마이크로 음성을 녹음한 후, Azure Blob Storage에 직접 업로드"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    blob_name = f"live_audio_{timestamp}.wav"
    blob_client = blob_service_client.get_blob_client(container=BLOB_AUDIO_CONTAINER, blob=blob_name)

    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

    print("녹음 시작... (Enter 또는 CTRL+C를 눌러 종료)")
    frames = []

    try:
        while True:
            data = stream.read(CHUNK)
            frames.append(data)
            if keyboard.is_pressed("enter"):
                print("녹음 종료")
                break
    except KeyboardInterrupt:
        print("녹음 종료")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    # **메모리에 WAV 파일 변환 (로컬 저장 X)**
    wav_data = io.BytesIO()
    with wave.open(wav_data, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(2)  # **16비트 PCM**
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"Blob Storage에 업로드 중: {blob_name}")
    blob_client.upload_blob(wav_data.getvalue(), overwrite=True)

    return blob_name


class BinaryAudioStreamReader(speechsdk.audio.PullAudioInputStreamCallback):
    """Azure Speech SDK에서 사용할 수 있도록 Binary Audio Stream Reader 정의"""
    def __init__(self, audio_data):
        super().__init__()
        self._audio_data = io.BytesIO(audio_data)

    def read(self, buffer):
        """Azure Speech SDK에서 호출하는 read 메서드"""
        size = len(buffer)
        data = self._audio_data.read(size)
        buffer[:len(data)] = data  # 메모리 버퍼에 데이터를 채움
        return len(data)

    def close(self):
        """스트림 종료"""
        self._audio_data.close()


def transcribe_audio(blob_name):
    """Azure STT API를 사용하여 Blob Storage에서 직접 변환"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    blob_client = blob_service_client.get_blob_client(container=BLOB_AUDIO_CONTAINER, blob=blob_name)
    blob_data = blob_client.download_blob().readall()  # **Blob Storage에서 WAV 파일 데이터 가져오기**

    speech_key = os.getenv("SPEECH_API_KEY")
    speech_region = os.getenv("SPEECH_REGION")

    if not speech_key or not speech_region:
        raise ValueError("환경 변수 SPEECH_API_KEY 또는 SPEECH_REGION이 설정되지 않았습니다.")

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language = "ko-KR"

    # ** WAV 데이터를 올바르게 변환하여 PullAudioInputStream 생성**
    stream_reader = BinaryAudioStreamReader(blob_data)
    audio_input_stream = speechsdk.audio.PullAudioInputStream(stream_reader)
    audio_config = speechsdk.audio.AudioConfig(stream=audio_input_stream)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("음성을 텍스트로 변환 중...")
    result = speech_recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print(f"STT 변환 결과: {result.text}")
        return result.text
    else:
        print("음성 인식 실패")
        return None



def upload_stt_result_to_blob(text_data, audio_blob_name):
    """STT 변환된 텍스트를 Blob Storage에 저장"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    text_blob_name = audio_blob_name.replace(".wav", ".txt")
    blob_client = blob_service_client.get_blob_client(container=STT_RESULTS_CONTAINER, blob=text_blob_name)
    blob_client.upload_blob(text_data, overwrite=True)
    print(f"STT 변환된 텍스트 업로드 완료: {text_blob_name}")
    return text_blob_name

def summarize_text(text):
    """Azure OpenAI를 사용하여 STT 결과를 요약"""
    prompt = f"""
    당신은 환자와 의사의 대화를 분석하는 AI입니다.  
    아래의 대화 내용을 기반으로 **핵심 정보는 유지하면서도 짧고 간결한 요약**을 생성하세요.  

    ❗ 단, 언급한 의학적 내용이 삭제되지 않도록 하며, 설명이 필요하면 간결한 문장으로 추가하세요.  
    ❗ 정보가 너무 축약되지 않도록 하며, **한 항목당 최소 2개 이상의 핵심 내용을 유지하세요.**  
    ❗ 줄글이 아닌, `진단:`, `수술/치료 계획:` 등의 키워드 형식을 유지하세요.  
    ❗ **부작용 및 위험** 항목에서는 주요 합병증(담관 손상, 담즙 누출, 출혈 등)에 대해 개별적으로 설명하세요. 이 부분만 줄글이고 나머지는 키워드 형식 유지.
    ❗ 환자가 우려하는 부분에 대해서는 반드시 요약하세요.

    입력 데이터:
    \"\"\" {text} \"\"\"  

    ### **출력 형식 (각 항목을 간결하지만 핵심 내용을 유지하면서 요약)**  
    - **진단:** [질병명]  
    - 예: 급성 담낭염  

    - **수술/치료 계획:** [필요한 절차, 치료 선택 이유, 치료 효과, 수술과 관련된 주요 문제점]  

    - **부작용 및 위험:** [예상 가능한 부작용, 합병증 발생 가능성, 합병증 발생 시 대처법]  
    - **[합병증명]:** [합병증 설명] 
    - **[합병증명]:** [합병증 설명] 

    - **주의사항:** [수술 전후 환자가 지켜야 할 사항, 생활습관 조정, 주의해야 할 합병증, 의료진에 알려야 하는 상황]  

    - **기타:** [환자의 질문과 답변, 기타 중요한 설명, 통증, 기저질환, 회복기간, 흉터에 대한 설명]  
    """

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "You are an AI assistant specialized in summarizing medical conversations."},
            {"role": "user", "content": prompt}
        ]
    )

    summary = response.choices[0].message.content.strip()
    return summary

def upload_summary_to_blob(summary_text, original_filename):
    """요약된 텍스트를 Blob Storage(sum_results 컨테이너)에 저장"""
    summary_blob_name = original_filename.replace(".txt", "_summary.txt")
    blob_client = blob_service_client.get_blob_client(container=SUMMARIZED_RESULTS_CONTAINER, blob=summary_blob_name)
    blob_client.upload_blob(summary_text, overwrite=True)
    print(f"요약된 텍스트 업로드 완료: {summary_blob_name}")

# 🔹 실행 흐름
audio_filename = record_and_upload()
stt_text = transcribe_audio(audio_filename)

if stt_text:
    stt_filename = upload_stt_result_to_blob(stt_text, audio_filename)
    summarized_text = summarize_text(stt_text)
    upload_summary_to_blob(summarized_text, stt_filename)

    print("\n요약된 내용:")
    print(summarized_text)


녹음 시작... (Enter 또는 CTRL+C를 눌러 종료)
녹음 종료
Blob Storage에 업로드 중: live_audio_20250204_095618.wav
음성을 텍스트로 변환 중...
STT 변환 결과: 그래서 녹음을 뭐 어쩌고저쩌고 이렇게 한 다음에.
STT 변환된 텍스트 업로드 완료: live_audio_20250204_095618.txt
요약된 텍스트 업로드 완료: live_audio_20250204_095618_summary.txt

요약된 내용:
죄송하지만, 제공된 정보를 기반으로 요약을 생성할 수 없습니다. 입력 데이터에 대화 내용이 포함되어 있지 않기 때문입니다. 올바른 대화 내용을 제공해 주시면, 처리해 드리겠습니다.
